In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
from tqdm import tqdm

from environment_data import Environment
from windfarm_state import Windfarm_state
from ship_plan import Ship_plan

from collections import OrderedDict

total_step_by_three_hour = int(20*365*(24/3))      ############ とりま5年分！
total_number_of_ships = 6

def sim(actor_i):
    
    prefix = "[I actor" + str(actor_i) + "] "
    print(prefix+"read environment ...")
    print(prefix+"construct windfarm_state ...")
    print(prefix+"construct ship_plan ...")

    wind_hist = np.load("data/wind_hist.npy")
    can_work = np.load("data/can_work.npy")
    broken_program = np.load("data/broken_program.npy")

    environment = Environment(data=(wind_hist, can_work, broken_program))
    windfarm_state = Windfarm_state(environment)
    ship_plan = Ship_plan(total_number_of_ships, environment, windfarm_state)
    
    need_inspection_list = []
    need_repair_list = []
    total_generated_power = []

    for t in range(total_step_by_three_hour):
        ship_plan.time_step(t)
        windfarm_state.time_step(t)

        need_inspection_list.append(sum(windfarm_state.check_need_inspection_all()))
        need_repair_list.append(sum(windfarm_state.check_need_repair_all()))
        total_generated_power.append(windfarm_state.total_calc_generated_kwh())

        need_repair_all = np.array(windfarm_state.check_need_repair_all())
        there_is_ship_all = np.array(windfarm_state.check_there_is_ship_all())
        time_from_last_inspection_all = np.array(windfarm_state.check_time_from_last_inspection_all())

        # 戦略3から引っぱってきてる
        w_repair = sum(need_repair_all & ~there_is_ship_all)/5 # max5かなっていう # どれだけ故障を許容するかが鍵？
        w_inspection = np.mean(time_from_last_inspection_all)/2160 # max2160かなっていう
        p_repair = w_repair / (w_repair + w_inspection + 1e-12)

        if t % (365*8) == 0:
            print(prefix+"t:",int(t/(365*8)), ", total_pow:", total_generated_power[-1])

    print(prefix+"total_calc_generated_kwh: {:,}".format(windfarm_state.total_calc_generated_kwh()))
    print(prefix+"total_driving_cost: {:,}".format(ship_plan.total_driving_cost))
    print(prefix+"repayment cost: {:,}".format(400000000 * total_number_of_ships))
    print(prefix+"total_profit: {:,}".format(windfarm_state.total_calc_generated_kwh() -
                          ship_plan.total_driving_cost - 400000000 *
                          total_number_of_ships))

/home/nama/.pyenv/versions/3.6.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
sim(0)

[I actor0] read environment ...
[I actor0] construct windfarm_state ...
[I actor0] construct ship_plan ...
[I actor0] t: 0 , total_pow: 0
[I actor0] t: 1 , total_pow: 243889358400
[I actor0] t: 2 , total_pow: 479776219200
[I actor0] t: 3 , total_pow: 718684099200
[I actor0] t: 4 , total_pow: 947144836800
[I actor0] t: 5 , total_pow: 1186780312800
[I actor0] t: 6 , total_pow: 1422122702400
[I actor0] t: 7 , total_pow: 1670132131200
[I actor0] t: 8 , total_pow: 1916405071200
[I actor0] t: 9 , total_pow: 2147444265600
[I actor0] t: 10 , total_pow: 2365822857600
[I actor0] t: 11 , total_pow: 2594779272000
[I actor0] t: 12 , total_pow: 2831800500000
[I actor0] t: 13 , total_pow: 3052336802400
[I actor0] t: 14 , total_pow: 3298544769600
[I actor0] t: 15 , total_pow: 3532245991200
[I actor0] t: 16 , total_pow: 3772133863200
[I actor0] t: 17 , total_pow: 4000090780800
[I actor0] t: 18 , total_pow: 4247602502400
[I actor0] t: 19 , total_pow: 4495837348800
[I actor0] total_calc_generated_kwh: 4,

In [3]:
N_actor = 4
import concurrent.futures
executor = concurrent.futures.ThreadPoolExecutor(max_workers=N_actor)

for i in range(N_actor):
    executor.submit(sim, i)

[I actor0] read environment ...
[I actor0] construct windfarm_state ...
[I actor0] construct ship_plan ...
[I actor1] read environment ...[I actor2] read environment ...
[I actor1] construct windfarm_state ...

[I actor2] construct windfarm_state ...
[I actor2] construct ship_plan ...
[I actor1] construct ship_plan ...
[I actor3] read environment ...
[I actor3] construct windfarm_state ...
[I actor3] construct ship_plan ...
[I actor0] t: 0 , total_pow: 0
[I actor2] t: 0 , total_pow:[I actor3] t: 0 , total_pow: 0
 0
[I actor1] t: 0 , total_pow: 0
[I actor2] t: 1 , total_pow: 243889358400
[I actor3] t: 1 , total_pow: 243889358400
[I actor1] t: 1 , total_pow: 243889358400
[I actor0] t: 1 , total_pow: 243889358400
[I actor2] t: 2 , total_pow: 479776219200
[I actor3] t: 2 , total_pow: 479776219200
[I actor0] t: 2 , total_pow: 479776219200
[I actor1] t: 2 , total_pow: 479776219200
[I actor3] t: 3 , total_pow: 718684099200
[I actor0] t: 3 , total_pow: 718684099200
[I actor2] t: 3 , total_pow: